# Overview
In this example we will optimize the energy density of a battery cell by fine-tuning the thickness of three components
1. The positive electrode (30 - 150 microns)
2. The negative electrode (30 - 150 microns)
3. The separator (8-15 microns)

- The simulations will be done on the [Battmo Server](https://github.com/OpenBattTools/BattMo-Server).
- The parameter suggestions will be done using this wrapper to call SDLabs. 

In [2]:
import sdlabs_wrapper.wrapper as at
import sdlabs_wrapper.models as at_models
import getpass
from typing import Dict
import requests
import random

In [3]:
%load_ext autoreload
%autoreload 2

## Battmo functions

In [4]:
# constants
BATTMO_URL= "https://api.battmo.open-semantic-lab.org/api/run/performance-spec"

Test battmo server by sending a sample request

In [5]:
def send_battmo_request(batt_suggestion:Dict[str,any],battmo_url=BATTMO_URL):
    print(f"Sending request to Battmo with {batt_suggestion}")
    resp = requests.post(battmo_url, json = batt_suggestion).json()
    print(f"Received response {resp}")
    return resp["result"]

In [6]:
send_battmo_request({    "geometry": {
                "format": "1D",
                    "faceArea": 0.0001,
                    "NegativeElectrode": {
                      "ActiveMaterial": {
                        "thickness": 15e-6
                      }
                    },
                    "PositiveElectrode": {
                      "ActiveMaterial": {
                        "thickness": 50e-6
                      }
                    },
                    "Electrolyte": {
                      "Separator": {
                        "thickness": 50e-6
                      }
                    }
    }
            })

Sending request to Battmo with {'geometry': {'format': '1D', 'faceArea': 0.0001, 'NegativeElectrode': {'ActiveMaterial': {'thickness': 1.5e-05}}, 'PositiveElectrode': {'ActiveMaterial': {'thickness': 5e-05}}, 'Electrolyte': {'Separator': {'thickness': 5e-05}}}}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.battmo.open-semantic-lab.org:443
DEBUG:urllib3.connectionpool:https://api.battmo.open-semantic-lab.org:443 "POST /api/run/performance-spec HTTP/1.1" 200 97
Received response {'status': 'ok', 'result': {'E': 3.0, 'energyDensity': 1031368648.2790359, 'energy': 11.860739455208918}}


{'E': 3.0, 'energyDensity': 1031368648.2790359, 'energy': 11.860739455208918}

## Atinary Wrapper
Atinary wrapper can be instantiated from a config file (see `./optimization_config.json`). 
We also need an api_key which you can access under your account at [home.atinary.com](home.atinary.com)
The wrapper uses the models available at `atinary/src/sdlabs_wrapper/models.py`. A JSON schema is available under `sdlabs_wrapper_schema.json`

In [7]:
api_key = getpass.getpass()

In [8]:
def transform_atinary_to_battmo(at_recommendation:at_models.Recommendation):
     return {    "geometry": {
                "format": "1D",
                    "faceArea": 0.0001,
                    "NegativeElectrode": {
                      "ActiveMaterial": {
                        "thickness": at_recommendation.param_values['neg_electrode_thickness']
                      }
                    },
                    "PositiveElectrode": {
                      "ActiveMaterial": {
                        "thickness": at_recommendation.param_values['pos_electrode_thickness']
                      }
                    },
                    "Electrolyte": {
                      "Separator": {
                        "thickness": at_recommendation.param_values['separator_thickness']
                      }
                    }
    }
            }

In [11]:
# look at config.json
!cat ./optimization_config.json

{
    "optimization_name": "BattmoSimulation New wrapper",
    "description": "Optimize cell layer thicknesses",
    "sdlabs_group_id": "onterface",
    "parameters": [
        {
            "low_value": 30e-6,
            "high_value": 150e-6,
            "name": "neg_electrode_thickness"
        },
        {
            "low_value": 30e-6,
            "high_value": 150e-6,
            "name": "pos_electrode_thickness"
        },
        {
            "low_value": 8e-6,
            "high_value": 15e-6,
            "name": "separator_thickness"
        }
    ],
    "multi_objective_function": "chimera",
    "objectives": [
        {
            "name": "energy_density",
            "goal": "maximize"
        }
    ],
    "inherit_data": false,
    "always_restart": false,
    "batch_size": 1,
    "algorithm": "dragonfly",
    "budget": 20
}

In [10]:
sdlabs_wrapper = at.initialize_optimization(
    inherit_data=False,
    always_restart=True,
    spec_file_path="./optimization_config.json",
    api_key=api_key,
)
print(f"Optimization loaded and started according to configuration file.")
for iteration in range(sdlabs_wrapper.config.budget):
    print("Waiting for new suggestions")
    suggestions = sdlabs_wrapper.get_new_suggestions(max_retries=10, sleep_time_s=15)
    print(f"Iteration {iteration+1} New Atinary suggestions: {suggestions}")
    for suggestion in suggestions:
        # transform to battmo format:
        print(suggestion.param_values)
        battmo_param = transform_atinary_to_battmo(suggestion)
        print(f"Battmo parameters: {battmo_param}")
        # Call Battmo here
        response = send_battmo_request(battmo_param)
        print(f"Battmo response {response}") 
        
        # measurements should be in the form of a dict with keys matching objectives and numerical values
        suggestion.measurements = {sdlabs_wrapper.config.objectives[0].name:response["energyDensity"]}
    if suggestions:
        print(f"Sending measurements back to SDLabs")
        sdlabs_wrapper.send_measurements(suggestions)

INFO:sdlabs_wrapper.wrapper:Creating new workstation associated to parameters [Parameter(name='neg_electrode_thickness', type='continuous', high_value=0.00015, low_value=3e-05, stride=None, description=None, descriptors=None), Parameter(name='pos_electrode_thickness', type='continuous', high_value=0.00015, low_value=3e-05, stride=None, description=None, descriptors=None), Parameter(name='separator_thickness', type='continuous', high_value=1.5e-05, low_value=8e-06, stride=None, description=None, descriptors=None)] and measurements ['energy_density']
INFO:sdlabs_wrapper.wrapper:Linked to workstation 'wst_bb7c0e99709c4a9e8dd5f13ab08d1c62'
INFO:sdlabs_wrapper.wrapper:New campaign launched! Preload data set to 'False'.
INFO:sdlabs_wrapper.wrapper:Running optimization associated with campaign id : 'cpg_758f9e51b6cb47fe99943b4930d0fa3b' associated to template 'tpl_77979453da0c49f1a64ecb089be7d597'
Optimization loaded and started according to configuration file.
Waiting for new suggestions
INF

You can now access the platform to create a custom report of your optimization under https://home.atinary.com Here is a sample demo of the optimization under the Analytics dashboard of SDLabs.
![Optimization plot](analytics_demo.png)